# Creating STAC items from large, remote netCDF4 files

It GOES, obviously.
GDAL only allows VSI virtual file system API support with new-ish linux kernels: https://gdal.org/drivers/raster/netcdf.html#vsi-virtual-file-system-api-support.
This makes testing/development on OSX and other systems a PITA.
But we're not actually reading the data, we just want to projection information and other tags.
Since netCDF4 files are based on HDF5, let's use HDF5 to read the information over the network.
Specifically, we'll grab projection information as described in https://gdal.org/drivers/raster/netcdf.html#georeference and create the spatial reference via pyproj.

In [1]:
import h5py
import fsspec

url = "https://noaa-goes16.s3.amazonaws.com/ABI-L2-CMIPM/2021/140/12/OR_ABI-L2-CMIPM1-M6C01_G16_s20211401200257_e20211401200315_c20211401200376.nc"
with fsspec.open(url) as bytes:
    file = h5py.File(bytes)
    projection = file["goes_imager_projection"]
    sweep_angle_axis = projection.attrs["sweep_angle_axis"].decode("utf-8")
    satellite_height = projection.attrs["perspective_point_height"][0]
    latitude_natural_origin = projection.attrs["latitude_of_projection_origin"][0]
    longitude_natrual_origin = projection.attrs["longitude_of_projection_origin"][0]
    extent = file["geospatial_lat_lon_extent"]
    xmin = extent.attrs["geospatial_westbound_longitude"][0]
    ymin = extent.attrs["geospatial_southbound_latitude"][0]
    xmax = extent.attrs["geospatial_eastbound_longitude"][0]
    ymax = extent.attrs["geospatial_northbound_latitude"][0]
    rowcount = len(file["x"][:])
    colcount = len(file["y"][:])
    x_bounds = file["x_image_bounds"][:]
    y_bounds = file["y_image_bounds"][:]
    for key in file.keys():
        print(key)

CMI
DQF
algorithm_dynamic_input_data_container
algorithm_product_version_container
band
band_id
band_wavelength
channel_gain_field
channel_integration_time
earth_sun_distance_anomaly_in_AU
esun
focal_plane_temperature_threshold_decreasing
focal_plane_temperature_threshold_exceeded_count
focal_plane_temperature_threshold_increasing
geospatial_lat_lon_extent
goes_imager_projection
kappa0
max_reflectance_factor
maximum_focal_plane_temperature
mean_reflectance_factor
min_reflectance_factor
nominal_satellite_height
nominal_satellite_subpoint_lat
nominal_satellite_subpoint_lon
number_of_image_bounds
number_of_time_bounds
outlier_pixel_count
percent_uncorrectable_GRB_errors
percent_uncorrectable_L0_errors
planck_bc1
planck_bc2
planck_fk1
planck_fk2
processing_parm_version_container
std_dev_reflectance_factor
t
time_bounds
total_number_of_points
valid_pixel_count
x
x_image
x_image_bounds
y
y_image
y_image_bounds


In [3]:
from pyproj.crs import ProjectedCRS, GeographicCRS
from pyproj.crs.datum import CustomDatum, CustomEllipsoid
from pyproj.crs.coordinate_operation import GeostationarySatelliteConversion

ellipsoid = CustomEllipsoid.from_name("GRS80")
datum = CustomDatum(ellipsoid=ellipsoid)
conversion = GeostationarySatelliteConversion(sweep_angle_axis, satellite_height, latitude_natural_origin, longitude_natrual_origin)
crs = ProjectedCRS(conversion=conversion, geodetic_crs=GeographicCRS(datum=datum))
projection_shape = [rowcount, colcount]
xres = (x_bounds[1] - x_bounds[0]) / rowcount
yres = (y_bounds[1] - y_bounds[0]) / colcount
projection_transform = [
    xres, 0, x_bounds[0], 0, -yres, y_bounds[1], 0, 0, 1
]

In [45]:
from pyproj import CRS

rio_crs = CRS("PROJCS[\"unnamed\",GEOGCS[\"unknown\",DATUM[\"unnamed\",SPHEROID[\"Spheroid\",6378137,298.2572221]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]]],PROJECTION[\"Geostationary_Satellite\"],PARAMETER[\"central_meridian\",-75],PARAMETER[\"satellite_height\",35786023],PARAMETER[\"false_easting\",0],PARAMETER[\"false_northing\",0],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],EXTENSION[\"PROJ4\",\"+proj=geos +lon_0=-75 +h=35786023 +x_0=0 +y_0=0 +ellps=GRS80 +units=m +no_defs +sweep=x\"]]")

In [49]:
from difflib import Differ
import sys

crs_json = crs.to_json(pretty=True).splitlines(keepends=True)
rio_crs_json = rio_crs.to_json(pretty=True).splitlines(keepends=True)
differ = Differ()
result = list(differ.compare(crs_json, rio_crs_json))
sys.stdout.writelines(result)

  {
    "$schema": "https://proj.org/schemas/v0.2/projjson.schema.json",
    "type": "ProjectedCRS",
-   "name": "undefined",
?              ----
+   "name": "unnamed",
?               ++
    "base_crs": {
-     "name": "undefined",
?                ^^^^ ^^
+     "name": "unknown",
?                ^ ^^^
      "datum": {
        "type": "GeodeticReferenceFrame",
-       "name": "undefined",
?                  ----
+       "name": "unnamed",
?                   ++
        "ellipsoid": {
-         "name": "GRS 1980(IUGG, 1980)",
+         "name": "Spheroid",
          "semi_major_axis": 6378137,
-         "inverse_flattening": 298.257222101
?                                          --
+         "inverse_flattening": 298.2572221
        }
      },
      "coordinate_system": {
        "subtype": "ellipsoidal",
        "axis": [
          {
            "name": "Longitude",
            "abbreviation": "lon",
            "direction": "east",
            "unit": "degree"
          },
        